In [1]:
import os
from datetime import datetime
from typing import Literal

import torch
import wandb
import numpy as np
from torch import nn
from torch import optim
import torch_geometric.nn as geonn
from tqdm import tqdm

from gfos.data.utils import load_layout
from gfos.data.dataset import LayoutDataset
from gfos.utils.scheduler import CosineAnnealingWarmupRestarts
from gfos.metrics import metric_for_layout_collections
from gfos.loss import MultiElementRankLoss, listMLE
from gfos.utils.misc import seed_everything


SEED = 42
DEBUG = False

seed_everything(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

ImportError: cannot import name 'metric_for_layout_collections' from 'gfos.metrics' (g:\projects\gfos\src\gfos\metrics.py)

## Configs

In [2]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"

configs = dict(
    conv_layer="GATConv",
    num_epochs=100,
    learning_rate=5e-4,
    weight_decay=1e-6,
    min_lr=1e-7,
    warmup_ratio=0.0,
    max_configs=80,
    graph_hidden=[16, 32, 16, 48, 64],
    num_encoder=1,
    num_feedforward=512,
    nhead=1,
    loss_margin=0.1,
    loss_num_permutations=2000,
)

WANDB_PROJECT = "gfos"
WANDB_DIR = "../../logs/"
WANDB_RUN_NAME = "gcn_layout_xla"
TAGS = ["train", "layout", "xla"]


## Model

In [3]:
# def transform_node_positional_embeddings(
#     embeddings_output: torch.Tensor,
#     node_config_ids: torch.Tensor,
#     num_nodes: int,
# ) -> torch.Tensor:
#     num_configs, _, dim = embeddings_output.shape
#     idxs = node_config_ids.unsqueeze(0).repeat(num_configs, 1) # [c, nc]
#     zeros = torch.zeros(
#         num_configs,
#         num_nodes,
#         dim,
#         device=embeddings_output.device,
#         dtype=embeddings_output.dtype,
#     )
#     idxs = idxs.unsqueeze(-1).repeat(1, 1, dim) # [c, nc, dim]
#     zeros.scatter_reduce_(1, idxs, embeddings_output, reduce="sum")
#     return zeros

In [4]:
def edges_adjacency(edges: torch.Tensor, add_diagonal=True) -> torch.Tensor:
    """
    Generate an adjacency matrix from the edges
    Args:
        edges: Tensor of shape (num_edges, 2) with the edges
        add_diagonal: Boolean indicating if the diagonal should be added to the adjacency matrix
    Returns:
        adjacency_matrix: Tensor of shape (num_nodes, num_nodes) with the adjacency matrix
    """
    adjacency_matrix = torch.zeros(
        (edges.max() + 1, edges.max() + 1), device=edges.device
    )
    adjacency_matrix[edges[:, 0], edges[:, 1]] = 1
    if add_diagonal:
        diag_idx = torch.arange(adjacency_matrix.shape[0])
        adjacency_matrix[diag_idx, diag_idx] = 1
    return adjacency_matrix


class LayoutModel(torch.nn.Module):
    def __init__(
        self,
        conv_layer: Literal["GATConv", "GCNConv", "SAGEConv"],
        hidden_channels: list[int],
        num_encoder: int = 1,
        num_feedforward: int = 256,
        nhead: int = 1,
    ):
        super().__init__()

        conv_layer = getattr(geonn, conv_layer)

        op_embedding_dim = 32
        config_dim = 64
        graph_out = hidden_channels[-1]
        merged_node_dim = graph_out + config_dim

        self.embedding = torch.nn.Embedding(
            120,
            op_embedding_dim,
        )
        assert len(hidden_channels) > 0
        in_channels = op_embedding_dim + 140
        self.convs = torch.nn.ModuleList()

        hidden_channels = [in_channels] + hidden_channels
        for i in range(len(hidden_channels) - 1):
            self.convs += [
                conv_layer(hidden_channels[i], hidden_channels[i + 1]),
            ]
            nn.LeakyReLU()

        # Transformer encoder to merge configs features with graph features
        layer = nn.TransformerEncoderLayer(
            d_model=merged_node_dim,
            dim_feedforward=num_feedforward,
            nhead=nhead,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers=num_encoder)

        self.layernorm = nn.LayerNorm(merged_node_dim)

        self.config_prj = nn.Sequential(
            nn.Linear(18, config_dim),
            nn.LayerNorm(config_dim),
            nn.LeakyReLU(),
        )

        # Define a sequential dense neural network
        self.dense = torch.nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(merged_node_dim, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 1),
        )

    def forward(
        self,
        node_feat: torch.Tensor,
        node_opcode: torch.Tensor,
        edge_index: torch.Tensor,
        node_config_feat: torch.Tensor,
        node_config_ids: torch.Tensor,
    ) -> torch.Tensor:
        # Get graph features
        c = node_config_feat.size(0)

        x = torch.cat([node_feat, self.embedding(node_opcode)], dim=1)

        # Get graph features
        for conv in self.convs:
            x = conv(x, edge_index)

        adjacency_matrix = edges_adjacency(edge_index, add_diagonal=False)
        neighbor_feat = (adjacency_matrix @ x) / (len(node_opcode) - 1)

        # (N, graph_out) -> (NC, graph_out)
        x = x[node_config_ids]
        neighbor_feat = neighbor_feat[node_config_ids]
        x += neighbor_feat

        # Merge graph features with config features
        # (C, NC, 18) -> (C, NC, config_dim)
        node_config_feat = self.config_prj(node_config_feat)

        # (C, NC, graph_out + config_dim)
        x = torch.cat([x.repeat((c, 1, 1)), node_config_feat], dim=-1)
        x = nn.functional.normalize(x, dim=-1)

        # (C, NC, graph_out + config_dim) -> (C, graph_out + config_dim)
        x = self.encoder(x)[:, -1, :]
        x = self.dense(x).flatten()

        return x

## Valid

In [5]:
layout_random = load_layout(
    LAYOUT_DIR,
    model_type="xla",
    compile_type="random",
)
val_dataset = LayoutDataset(files=layout_random["valid"], permute=False)
_INFERENCE_CONFIGS_BATCH_SIZE = 100

conv_layer = configs["conv_layer"]
num_epochs = configs["num_epochs"]
learning_rate = configs["learning_rate"]
weight_decay = configs["weight_decay"]
min_lr = configs["min_lr"]
warmup_ratio = configs["warmup_ratio"]
max_configs = configs["max_configs"]
graph_hidden = configs["graph_hidden"]
num_encoder = configs["num_encoder"]
num_feedforward = configs["num_feedforward"]
nhead = configs["nhead"]
margin = configs["loss_margin"]
number_permutations = configs["loss_num_permutations"]
_INFERENCE_CONFIGS_BATCH_SIZE = 100

model = LayoutModel(
    conv_layer=conv_layer,
    hidden_channels=graph_hidden,
    num_encoder=num_encoder,
    num_feedforward=num_feedforward,
    nhead=nhead,
).to(device)

static_dict = torch.load("../../logs/gcn_layout_xla_random/20230928_025033/78_0.0130.pth", map_location=device)
model.load_state_dict(static_dict)
model = model.eval()

In [6]:
layout_xla_scores = []
pred_idx = []


with torch.no_grad():
    for data in tqdm(val_dataset):
        node_feat = data["node_feat"]
        node_opcode = data["node_opcode"]
        edge_index = data["edge_index"]
        node_config_feat = data["node_config_feat"]
        node_config_ids = data["node_config_ids"]
        target = data["target"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
        )

        target = target.numpy()
        num_configs = target.shape[-1]
        outs = []

        for i in range(0, num_configs, _INFERENCE_CONFIGS_BATCH_SIZE):
            end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
            out: torch.Tensor = model(
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat[i: end_i],
                node_config_ids,
            )
            outs.append(out.detach().cpu())
            pred_idx.append(np.argsort(out.detach().cpu().numpy()))

        score = metric_for_layout_collections(np.argsort(torch.concat(outs).numpy()), np.argsort(target))
        layout_xla_scores.append(score)


100%|██████████| 7/7 [00:17<00:00,  2.43s/it]


In [7]:
np.mean(layout_xla_scores)

0.012966885305219717